In [4]:
include("goddard_hermite-Simpson.jl")
include("goddard.jl")
include("goddard_Euler.jl")
using NLPModels
using NLPModelsIpopt
using NLPModelsJuMP
using SparseArrays
using SuiteSparse

In [ ]:
using Plots
function normalize_matrix(mat)
    min_val = minimum(mat)
    max_val = maximum(mat)
    (mat .- min_val) ./ (max_val - min_val)
end

function present_Heatmap(Mat,name="")
    norm_matrix = normalize_matrix(Mat)
    max_Mat = round(maximum(Mat),sigdigits = 4)
    min_Mat = round(minimum(Mat),sigdigits = 4)
    Lims = "[$min_Mat,$max_Mat]"
    heatmap(norm_matrix, color=:grays, 
            #clims =(min_Mat,max_Mat) ,
            xlabel="Columns", ylabel="Rows", title="Matrix Heatmap $name $Lims")
end

In [ ]:
nh = 100
model = rocket_model(nh)
nlp = MathOptNLPModel(model)
# Solve with Ipopt
res = ipopt(nlp)

# Get primal-dual solution
x = res.solution
y = res.multipliers

n = NLPModels.get_nvar(nlp)
m = NLPModels.get_ncon(nlp)

# Build sparse Hessian 
nnzh = NLPModels.get_nnzh(nlp)
Wi, Wj = NLPModels.hess_structure(nlp)
Wz = zeros(nnzh)
NLPModels.hess_coord!(nlp, x, y, Wz)
W = sparse(Wi, Wj, Wz, n, n)

# Build sparse Jacobian 
nnzj = NLPModels.get_nnzj(nlp)
Ji, Jj = NLPModels.jac_structure(nlp)
Jz = zeros(nnzj)
NLPModels.jac_coord!(nlp, x, Jz)
J = sparse(Ji, Jj, Jz, m, n)
j1 = Matrix(J)

In [ ]:
F = SparseArrays.SPQR.qr(J)
print(F.Q * F.R == J)
present_Heatmap(F.Q * F.R)

In [ ]:
W

In [ ]:
using LinearAlgebra
Z = nullspace(j1)

In [ ]:
H = Z'*Symmetric(W, :L)*Z
present_Heatmap(H,"H")

In [ ]:
J* Z

In [ ]:
eigvals(H)